In [16]:
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
import zipfile
import cv2
import random
from tqdm.notebook import tqdm
import shutil
from PIL import Image, ImageEnhance, ImageOps
import albumentations as A

In [17]:
# Create directories to store results
def setup_directories():
    base_dir = "/content/augmented_dataset"
    original_dir = os.path.join(base_dir, "original")
    augmented_dir = os.path.join(base_dir, "augmented")

    # Create directories if they don't exist
    for dir_path in [base_dir, original_dir, augmented_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)

    return base_dir, original_dir, augmented_dir

In [18]:
print("Please upload your Cowpox dataset zip file")
uploaded = files.upload()

Please upload your Cowpox dataset zip file


Saving Cowpox.zip to Cowpox.zip


In [19]:
# Create a function to extract the uploaded zip file
def extract_uploaded_zip():
    # Get the first uploaded file (assuming it's the zip file)
    zip_filename = list(uploaded.keys())[0]

    # Extract the zip file
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall('/content/dataset')

    print(f"Extracted {zip_filename} to /content/dataset")

    # Return the path to the extracted dataset
    return '/content/dataset'

In [20]:
# Define augmentation functions
def apply_augmentations(image, augmentation_types):
    """
    Apply selected augmentations to an image

    Parameters:
    - image: Input image
    - augmentation_types: List of augmentation types to apply

    Returns:
    - List of augmented images
    """
    augmented_images = []

In [27]:
import cv2
import albumentations as A

def apply_augmentations(image, augmentation_types):
    """
    Apply selected augmentations to an image

    Parameters:
    - image: Input image
    - augmentation_types: List of augmentation types to apply

    Returns:
    - List of augmented images as (augmentation_name, image) tuples
    """
    augmented_images = []
    for aug_type in augmentation_types:
        if aug_type == 'horizontal_flip':
            aug_img = cv2.flip(image.copy(), 1)
            augmented_images.append(('horizontal_flip', aug_img))

        elif aug_type == 'vertical_flip':
            aug_img = cv2.flip(image.copy(), 0)
            augmented_images.append(('vertical_flip', aug_img))

        elif aug_type == 'brightness_increase':
            transform = A.Compose([
                A.RandomBrightnessContrast(brightness_limit=(0.2, 0.2), contrast_limit=0, p=1)
            ])
            aug_img = transform(image=image)['image']
            augmented_images.append(('brightness_increase', aug_img))

        elif aug_type == 'brightness_decrease':
            transform = A.Compose([
                A.RandomBrightnessContrast(brightness_limit=(-0.2, -0.2), contrast_limit=0, p=1)
            ])
            aug_img = transform(image=image)['image']
            augmented_images.append(('brightness_decrease', aug_img))

        elif aug_type == 'contrast_increase':
            transform = A.Compose([
                A.RandomBrightnessContrast(brightness_limit=0, contrast_limit=(0.2, 0.2), p=1)
            ])
            aug_img = transform(image=image)['image']
            augmented_images.append(('contrast_increase', aug_img))

        elif aug_type == 'hue_shift':
            transform = A.Compose([
                A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=0, val_shift_limit=0, p=1)
            ])
            aug_img = transform(image=image)['image']
            augmented_images.append(('hue_shift', aug_img))

        elif aug_type == 'saturation_increase':
            transform = A.Compose([
                A.HueSaturationValue(hue_shift_limit=0, sat_shift_limit=(30, 30), val_shift_limit=0, p=1)
            ])
            aug_img = transform(image=image)['image']
            augmented_images.append(('saturation_increase', aug_img))

        elif aug_type == 'gaussian_noise':
            transform = A.Compose([
                A.GaussNoise(var_limit=(10, 50), p=1)
            ])
            aug_img = transform(image=image)['image']
            augmented_images.append(('gaussian_noise', aug_img))

        elif aug_type == 'salt_pepper_noise':
            transform = A.Compose([
                A.ISONoise(color_shift=(0.01, 0.05), intensity=(0.1, 0.5), p=1)
            ])
            aug_img = transform(image=image)['image']
            augmented_images.append(('salt_pepper_noise', aug_img))

        elif aug_type == 'zoom_in':
            h, w = image.shape[:2]
            crop_size = int(min(h, w) * 0.8)  # 80% crop
            x = (w - crop_size) // 2
            y = (h - crop_size) // 2
            cropped = image[y:y+crop_size, x:x+crop_size]
            aug_img = cv2.resize(cropped, (w, h))
            augmented_images.append(('zoom_in', aug_img))

    return augmented_images


In [28]:
# Function to process the dataset
def process_dataset(dataset_path, original_dir, augmented_dir, augmentation_types):
    """
    Process the dataset by applying augmentations to all images

    Parameters:
    - dataset_path: Path to the dataset
    - original_dir: Directory to save original images
    - augmented_dir: Directory to save augmented images
    - augmentation_types: List of augmentation types to apply
    """
    # Check if dataset_path is a directory or a file path pattern
    if os.path.isdir(dataset_path):
        # It's a directory, get all image files
        image_paths = []
        valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

        # Walk through the directory
        for root, _, files in os.walk(dataset_path):
            for file in files:
                if any(file.lower().endswith(ext) for ext in valid_extensions):
                    image_paths.append(os.path.join(root, file))
    else:
        # It might be a specific file or a pattern
        print("The provided path is not a directory.")
        return

    print(f"Found {len(image_paths)} images in the dataset")

    # Process each image
    for img_path in tqdm(image_paths, desc="Processing images"):
        try:
            # Read the image
            image = cv2.imread(img_path)
            if image is None:
                print(f"Could not read image: {img_path}")
                continue

            # Convert BGR to RGB (OpenCV loads images as BGR)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Get the filename without extension
            filename = os.path.basename(img_path)
            name, ext = os.path.splitext(filename)

            # Save the original image
            original_img_path = os.path.join(original_dir, filename)
            cv2.imwrite(original_img_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

            # Apply augmentations
            augmented_images = apply_augmentations(image, augmentation_types)

            # Save augmented images
            for aug_type, aug_img in augmented_images:
                aug_filename = f"{name}_{aug_type}{ext}"
                aug_img_path = os.path.join(augmented_dir, aug_filename)
                cv2.imwrite(aug_img_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

        except Exception as e:
            print(f"Error processing {img_path}: {str(e)}")

    print(f"Augmentation complete! Augmented images saved to {augmented_dir}")


In [29]:
# Function to visualize sample augmentations
def visualize_augmentations(dataset_path, augmentation_types):
    """
    Visualize sample augmentations on a random image from the dataset

    Parameters:
    - dataset_path: Path to the dataset
    - augmentation_types: List of augmentation types to apply
    """
    # Get all image files
    image_paths = []
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

    # Walk through the directory
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in valid_extensions):
                image_paths.append(os.path.join(root, file))

    if not image_paths:
        print("No images found in the dataset")
        return


In [31]:
# Function to visualize sample augmentations
def visualize_augmentations(dataset_path, augmentation_types):
    """
    Visualize sample augmentations on a random image from the dataset

    Parameters:
    - dataset_path: Path to the dataset
    - augmentation_types: List of augmentation types to apply
    """
    # Get all image files
    image_paths = []
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp']

    # Walk through the directory
    for root, _, files in os.walk(dataset_path):
        for file in files:
            if any(file.lower().endswith(ext) for ext in valid_extensions):
                image_paths.append(os.path.join(root, file))

    if not image_paths:
        print("No images found in the dataset")
        return

    # Select a random image  # This line was incorrectly indented
    img_path = random.choice(image_paths)
    image = cv2.imread(img_path)
    if image is None:
        print(f"Could not read image: {img_path}")
        return

    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Apply augmentations
    augmented_images = apply_augmentations(image, augmentation_types)

    # Calculate grid size
    n_images = len(augmented_images) + 1  # +1 for the original image
    grid_size = int(np.ceil(np.sqrt(n_images)))

    # Create figure
    plt.figure(figsize=(15, 15))

    # Plot original image
    plt.subplot(grid_size, grid_size, 1)
    plt.title("Original")
    plt.imshow(image)
    plt.axis('off')

    # Plot augmented images
    for i, (aug_type, aug_img) in enumerate(augmented_images, start=2):
        plt.subplot(grid_size, grid_size, i)
        plt.title(aug_type)
        plt.imshow(aug_img)
        plt.axis('off')

    plt.tight_layout()
    plt.show()

In [32]:
# Function to compress the augmented dataset
def compress_dataset(base_dir):
    """
    Compress the augmented dataset into a zip file

    Parameters:
    - base_dir: Base directory containing the augmented dataset

    Returns:
    - Path to the zip file
    """
    zip_path = os.path.join("/content", "augmented_cowpox_dataset.zip")

    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(base_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, os.path.dirname(base_dir))
                zipf.write(file_path, arcname)

    print(f"Dataset compressed to {zip_path}")
    return zip_path

In [33]:
# Function to download the augmented dataset
def download_dataset(zip_path):
    """
    Download the compressed dataset

    Parameters:
    - zip_path: Path to the zip file
    """
    files.download(zip_path)

# Main execution
# Set up directories
base_dir, original_dir, augmented_dir = setup_directories()

# Run once to extract uploaded dataset
# Comment this out if you're uploading files manually
try:
    dataset_path = extract_uploaded_zip()
except Exception as e:
    print(f"Error extracting zip file: {str(e)}")
    print("Please upload your dataset manually to /content/dataset")
    dataset_path = '/content/dataset'


Extracted Cowpox.zip to /content/dataset


In [34]:
# Define augmentation types to apply
augmentation_types = [
    'horizontal_flip',
    'vertical_flip',
    'brightness_increase',
    'brightness_decrease',
    'contrast_increase',
    'hue_shift',
    'saturation_increase',
    'gaussian_noise',
    'salt_pepper_noise',
    'zoom_in',
    'zoom_out'
]

In [35]:
# Process the dataset
process_dataset(dataset_path, original_dir, augmented_dir, augmentation_types)

# Compress the augmented dataset
zip_path = compress_dataset(base_dir)

# Download the augmented dataset
print("Downloading the augmented dataset...")
download_dataset(zip_path)
print("Download complete!")

Found 88 images in the dataset


Processing images:   0%|          | 0/88 [00:00<?, ?it/s]

<ipython-input-27-c1e8de8e5a7d>:62: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10, 50), p=1)


Augmentation complete! Augmented images saved to /content/augmented_dataset/augmented
Dataset compressed to /content/augmented_cowpox_dataset.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download complete!
